In [ ]:
import cameratransform as ct
import cv2
import numpy as np
from typing import Dict

# The code below assumes that all cameras are located in the exact same location

geo_ref_points = [
    (52.42423212970922, 10.787014633845363, 0),
    (52.42436928364542, 10.787181148414293, 0),
    (52.42436089359796, 10.787262239475789, 0),
    (52.424514136503404, 10.787046448958769, 0),
    (52.424198163554216, 10.78707397280349, 0),
    (52.424235206487246, 10.786827161718584, 0),
    (52.42458243621526, 10.786827461813349, 0),
    (52.424136015933435, 10.786728569238377, 0),
    (52.424383772112314, 10.786863438269426, 0)
]

camera_params = {
    "244": {
        "elevation_m": 10,
        "heading_deg": 37,
        "image_height_px": 960,
        "image_width_px": 1280,
        "pos_lat": 52.42415872,
        "pos_lon": 10.78691456,
        "tilt_deg": 46.3,
        "roll_deg": -1.5,
        "view_x_deg": 80.4,
        "view_y_deg": None,
        "brown_distortion_k1": -0.225,
        "brown_distortion_k2": 0.035,
    },
    "238": {
        "elevation_m": 10,
        "heading_deg": 322,
        "image_height_px": 960,
        "image_width_px": 1280,
        "pos_lat": 52.42415872,
        "pos_lon": 10.78691456,
        "tilt_deg": 46.9,
        "roll_deg": 3.8,
        "view_x_deg": 84.5,
        "view_y_deg": None,
        "brown_distortion_k1": -0.225,
        "brown_distortion_k2": 0.035,
    }
}

cameras: Dict[str, ct.Camera] = {}

top_views = {}

for name, params in camera_params.items():
    cam = ct.Camera(
        projection=ct.RectilinearProjection(
            image_height_px=params.get('image_height_px', None),
            image_width_px=params.get('image_width_px', None),
            view_x_deg=params.get('view_x_deg', None),
            view_y_deg=params.get('view_y_deg', None),
        ),
        orientation=ct.SpatialOrientation(
            elevation_m=params.get('elevation_m', None),
            heading_deg=params.get('heading_deg', None),
            tilt_deg=params.get('tilt_deg', None),
            roll_deg=params.get('roll_deg', None),
        ),
        lens=ct.BrownLensDistortion(
            k1=params.get('brown_distortion_k1', None),
            k2=params.get('brown_distortion_k2', None),
            k3=params.get('brown_distortion_k3', None),
        )
    )
    cam.setGPSpos(params['pos_lat'], params['pos_lon'])

    cameras[name] = cam

    img = cv2.imread(f'{name}_original.jpg', cv2.IMREAD_COLOR)
    
    top_view = cam.getTopViewOfImage(
        img,
        extent=(-100,100,-100,100),
        scaling=0.05
    )
    top_views[name] = top_view

    cv2.imwrite(f'{name}_ct_undistort.jpg', cam.undistortImage(img, extent=[-1000,2000,-1000,2000]))
    cv2.imwrite(f'{name}_ct_mapped.jpg', top_view)

ref_cam = next(iter(cameras.values()))
space_ref_points = ref_cam.spaceFromGPS(np.array(geo_ref_points))
space_ref_points[:,1] *= -1
space_ref_points[:,:2] += 100
space_ref_points[:,:2] *= 20

merged = next(iter(top_views.values()))
for top_view in list(top_views.values())[1:]:
    merged = cv2.addWeighted(merged, 0.5, top_view, 0.5, 0.0)

diffed = next(iter(top_views.values()))
for top_view in list(top_views.values())[1:]:
    diffed = cv2.absdiff(diffed, top_view)

for point in space_ref_points:
    merged = cv2.circle(merged, (int(point[0]), int(point[1])), 8, color=(0, 0, 255), thickness=-1)
    diffed = cv2.circle(diffed, (int(point[0]), int(point[1])), 8, color=(0, 0, 255), thickness=-1)

cv2.imwrite(f'merged.jpg', merged)
cv2.imwrite(f'diffed.jpg', diffed)
